<a href="https://colab.research.google.com/github/sayakpaul/MIRNet-TFLite/blob/main/Add_Metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Reference: https://github.com/margaretmz/selfie2anime-with-tflite/blob/master/ml/add-meta-data-Colab/Add%20metadata%20to%20selfie2anime.ipynb. 
- TensorFlow Lite meatdata: https://www.tensorflow.org/lite/convert/metadata.
- Authored by: Sayak.
- Updated on: December 05 2020.

In [ ]:
!pip install -q tflite-support

     |████████████████████████████████| 1.0MB 5.6MB/s 
     |████████████████████████████████| 194kB 12.5MB/s 


In [ ]:
import os
import tensorflow as tf
from absl import flags

from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

print(tf.__version__)

2.3.0


In [ ]:
!mkdir model_without_metadata
!mkdir model_with_metadata

In [ ]:
!wget -q https://github.com/sayakpaul/MIRNet-TFLite/releases/download/v0.1.0/fixed_shape.zip
!unzip -q fixed_shape.zip

!mv fixed_shape/*.tflite model_without_metadata/

In [ ]:
# This is where we will export a new .tflite model file with metadata, and a .json file with metadata info
EXPORT_DIR = "model_with_metadata"

In [ ]:
class MetadataPopulatorForMIRNet(object):
  """Populates the metadata for the MIRNet model."""

  def __init__(self, model_file):
    self.model_file = model_file
    self.metadata_buf = None

  def populate(self):
    """Creates metadata and then populates it for a low-light image enhancement model."""
    self._create_metadata()
    self._populate_metadata()
  
  def _create_metadata(self):
    """Creates the metadata for the MIRNet model."""

    # Creates model info.
    model_meta = _metadata_fb.ModelMetadataT()
    model_meta.name = "MIRNet" 
    model_meta.description = ("Enhances a low-light image. Reference: https://arxiv.org/pdf/2003.06792v2.pdf. TFLiteConverter used from TensorFlow 2.3.0.")
    model_meta.version = "v1"
    model_meta.author = "Sayak"
    model_meta.license = ("Apache License. Version 2.0 "
                          "http://www.apache.org/licenses/LICENSE-2.0.")

    # Creates info for the input, normal image.
    input_image_meta = _metadata_fb.TensorMetadataT()
    input_image_meta.name = "low_light_image"
    # if self.model_type=="other":
    input_image_meta.description = (
            "The expected image is 400 x 400, with three channels "
            "(red, blue, and green) per pixel. Each value in the tensor is between"
            " 0 and 1.")
    input_image_meta.content = _metadata_fb.ContentT()
    input_image_meta.content.contentProperties = (
        _metadata_fb.ImagePropertiesT())
    input_image_meta.content.contentProperties.colorSpace = (
        _metadata_fb.ColorSpaceType.RGB)
    input_image_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.ImageProperties)
    input_image_normalization = _metadata_fb.ProcessUnitT()
    input_image_normalization.optionsType = (
        _metadata_fb.ProcessUnitOptions.NormalizationOptions)
    input_image_normalization.options = _metadata_fb.NormalizationOptionsT()
    input_image_normalization.options.mean = [0.0]
    input_image_normalization.options.std = [255.0]
    input_image_meta.processUnits = [input_image_normalization]
    input_image_stats = _metadata_fb.StatsT()
    input_image_stats.max = [1.0]
    input_image_stats.min = [0.0]
    input_image_meta.stats = input_image_stats


    # Creates output info, enhanced image
    output_image_meta = _metadata_fb.TensorMetadataT()
    output_image_meta.name = "enhanced_image"
    output_image_meta.description = "Image enhanced."
    output_image_meta.content = _metadata_fb.ContentT()
    output_image_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
    output_image_meta.content.contentProperties.colorSpace = (
        _metadata_fb.ColorSpaceType.RGB)
    output_image_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.ImageProperties)
    output_image_normalization = _metadata_fb.ProcessUnitT()
    output_image_normalization.optionsType = (
        _metadata_fb.ProcessUnitOptions.NormalizationOptions)
    output_image_normalization.options = _metadata_fb.NormalizationOptionsT()
    output_image_normalization.options.mean = [0.0]
    output_image_normalization.options.std = [1.0] 
    output_image_meta.processUnits = [output_image_normalization]
    output_image_stats = _metadata_fb.StatsT()
    output_image_stats.max = [255.0]
    output_image_stats.min = [0.0]
    output_image_meta.stats = output_image_stats

    # Creates subgraph info.
    subgraph = _metadata_fb.SubGraphMetadataT()
    subgraph.inputTensorMetadata = [input_image_meta] 
    subgraph.outputTensorMetadata = [output_image_meta] 
    model_meta.subgraphMetadata = [subgraph]

    b = flatbuffers.Builder(0)
    b.Finish(
        model_meta.Pack(b),
        _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
    self.metadata_buf = b.Output()

  def _populate_metadata(self):
    """Populates metadata to the model file."""
    populator = _metadata.MetadataPopulator.with_model_file(self.model_file)
    populator.load_metadata_buffer(self.metadata_buf)
    populator.populate()

In [ ]:
def populate_metadata(model_file):
  """Populates the metadata using the populator specified.
  Args:
      model_file: valid path to the model file.
  """

  # Populates metadata for the model.
  model_file_basename = os.path.basename(model_file)
  export_path = os.path.join(EXPORT_DIR, model_file_basename)
  tf.io.gfile.copy(model_file, export_path, overwrite=True)

  populator = MetadataPopulatorForMIRNet(export_path) 
  populator.populate()

  # Displays the metadata that was just populated into the tflite model.
  displayer = _metadata.MetadataDisplayer.with_model_file(export_path)
  export_json_file = os.path.join(
      EXPORT_DIR,
      os.path.splitext(model_file_basename)[0] + ".json")
  json_file = displayer.get_metadata_json()
  with open(export_json_file, "w") as f:
    f.write(json_file)
  print("Finished populating metadata and associated file to the model:")
  print(export_path)
  print("The metadata json file has been saved to:")
  print(os.path.join(EXPORT_DIR,
                   os.path.splitext(model_file_basename)[0] + ".json"))

In [ ]:
quantization = "int8" #@param ["dr", "int8", "fp16"]
tflite_model_path = f"mirnet_{quantization}.tflite" 
MODEL_FILE = "/content/model_without_metadata/{}".format(tflite_model_path)
populate_metadata(MODEL_FILE)

Finished populating metadata and associated file to the model:
model_with_metadata/mirnet_int8.tflite
The metadata json file has been saved to:
model_with_metadata/mirnet_int8.json


In [ ]:
!tar cvf model_with_metadata.tar.gz model_with_metadata

model_with_metadata/
model_with_metadata/mirnet_int8.tflite
model_with_metadata/mirnet_int8.json
model_with_metadata/mirnet_fp16.json
model_with_metadata/mirnet_fp16.tflite
model_with_metadata/mirnet_dr.json
model_with_metadata/mirnet_dr.tflite
